![Project Logo](Project_Logo.pdf)

# PyULN: Initial Condition Compiler
## Build 0.22 (31 Oct 2020) 

### Requires PyUL Helper Version 1+

### This is a Public Release
* Y. Wang: [yourong.f.wang@auckland.ac.nz]

* J. Zagorac
* R. Easther


## PyUltraLight 
### Now with N-Body Evalutation and Padded Potentials (H-E-B Scheme) and 2nd-Order Error Management

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Loading Required Packages

In [ ]:
import PyUltraLight_NBody_6 as PyUL
print('PyUltralight: Integrator Date', PyUL.D_version)

import PyUL_Helper as PyULH

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec
import matplotlib.animation
import math
import pyfftw
import os
import sys
import multiprocessing
import numpy
import numba
import h5py
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from IPython.core.display import clear_output, display

# Disk Space Management:

In [ ]:
save_path = 'FW_NBody_X'  # Set output directory

PyULH.DSManagement(save_path)

# ULDM Simulation Parameters

In [ ]:
##================================================================
## Important Ones

# Space Settings
resol = 96 
length = 12 # 1 code unit is ~38 kpc x (1e-22/m_a)^0.5
length_units = ''  # Accepted units: 'm', 'km', 'pc', 'kpc', 'Mpc', 'ly', and '' for dimensionless units.

# Time Settings
duration = 4 #1 code unit is ~70 Gyr (independent of axion mass assumption)
duration_units = ''  # Accepted units: 's', 'yr', 'kyr', 'Myr', and '' for dimensionless units
start_time = 0. # Should be given in the same units as duration. 
step_factor = 1 # Change this to a larger number if velocities are sufficiently low that constraint on timestep can be relaxed. 

##================================================================
## Other Flags
# Set units for soliton parameters
s_mass_unit = ''     #Accepted units: 'kg', 'solar_masses', 'M_solar_masses', and '' for dimensionless units
s_position_unit = '' #Accepted units: 'm', 'km', 'pc', 'kpc', 'Mpc', 'ly', and '' for dimensionless units
s_velocity_unit = '' #Accepted units: 'm/s', 'km/s', 'km/h', and '' for dimensionless units

# Rules ditto.
m_mass_unit = ''
m_position_unit = ''
m_velocity_unit = ''


##================================================================
## Multithreading, IO, and Control
num_threads = multiprocessing.cpu_count()
print("PyUL NBody: Available CPU threads for this run: ",num_threads)


print("PyUL NBody: Simulation Grid Size:", resol, 'cubed.')

LengthFS = PyUL.convert(length, length_units, 'l')

GridLenFS = LengthFS/(resol)

GridLenPC = PyUL.convert_back(GridLenFS,'pc','l')

print("PyUL NBody: The grid resolution subdivision is %.6f in specified units. This is %.6f code units." % ( length/resol, GridLenFS))


#Formats to save
hdf5 = False
npz = False
npy = True


save_number = 160 # Choose number of 'frames' to save. Note that, depending on resolution, this could require significant disk space.


#Data to save
#0
save_rho = False # Saves density data for entire 3D simulation grid
#1
save_psi = False # Saves full complex field data for entire 3D simulation grid
#2
save_plane = True # Saves density data for plane z = 0
#3
save_energies = True # Saves integrated gravitational, kinetic and total energies as lists
#4
save_line = False # Saves density data for line y = 0, z = 0. Useful for examining intereference patterns. 

## FW
#5
save_testmass = True # Saves trajectory and 3-velocity of a test mass thrown into the system.   
#6
save_phi = False
#7
save_phi_plane = True
#8
save_gradients = True


save_options = [save_rho,save_psi,save_plane,save_energies,save_line,save_testmass,save_phi,save_phi_plane,save_gradients]

SaveSizeEstimate = PyULH.SSEst(save_options,save_number,resol)

print('PyUL NBody: Estimated Save Size (GiB): %.02f' % SaveSizeEstimate )

# Data_Loader For Later




# Gravitational Field Smoothing
Instead of coding in the idealistic $\frac{m}{r}$ potential, we approximate it with $\frac{am}{ar+e^{-ar}}$. 

The modified potential is a useful approximation of a point mass in a distance, and puts an exponential cap on the infinite potential well, which reduces the impact of numerical artefacts.


The smoothing factor, $a$, is also defined here.


The differences between the two potentials are characterised in the following box.

In [ ]:
a = (2-0.15859433956303937)/length*resol

PyULH.SmoothingReport(a,resol,LengthFS)

# Load Example Scenarios:

Just run the following blocks, and "False" scenarios will automatically skip.

Both Parabola and Circular Orbit scenarios come with a great level of customizability, explained below. You can specify the initial positions, masses, and degree of ULDM participation in the dynamics.

If you want to specify your own settings entirely, please set both to False.




## - Parabola 

Describes two masses (soliton or BH or hybrid) with equal mass approaching each other on the $x-y$ plane.

You can specify the $(x_0,y_0)$ value, as well as their masses.

The rest of the initial conditions will then be automatically computed.


## - Circular Orbit 

Describes two masses (soliton or BH or hybrid) with arbitrary masses orbiting each other on the $x-y$ plane.

You can specify the initial separation, as well as their individual masses.

The rest of the initial conditions will then be automatically computed.

## - Tangyuan

This playful new addition has 3 solitons on -x, +y, and -y, and one mass at +x, all equal distance from each other and travels in the same circular orbit around their common centre of mass.
 
## - Collision

Specify your own impact parameter and center of mass location, and the program will try to make sure that the centre of mass stays at rest.

In [ ]:
############ PRE-DEFINED MODELS. YOU WILL BE PROMPTED TO SPECIFY THE CUSTOMIZABLE PARAMETERS FOR EACH

Parabola = False
Circular = False
Tangyuan = False
Collision = True

Settings = [Parabola,Circular,Tangyuan,Collision]

############ CREATE YOUR OWN INITIAL CONDITIONS HERE 

#Particle parameters are mass, position, and velocity
#Soliton parameters are mass, position, velocity, and phase (radians)

m1 = 0
mS = 2

BH1 = [m1,[3,0,0],[0,np.sqrt(mS/3),0]]


BHL1 = [8.0, [-0.08333333333333333, -0.5555555555555556, 0], [0, 0.1111111111111111, 0]]
BHL2 = [1.0, [0.6666666666666666, 4.444444444444445, 0], [0, -0.8888888888888888, 0]]

CustomParticles = []

soliton1 = [mS, [0,0,0], [5,5,0], 0]

CustomSolitons = [soliton1]  

############ LOADS DEMO SCENES INTO SYSTEM

particles, solitons = PyULH.Init(Settings,CustomParticles,CustomSolitons,resol,LengthFS)


########### MODIFIER PARAMETERS

# Removes Solitons in the IV and replaces them with a uniform wavefunction 
# with given probability amplitude (code unit).

Uniform = False
Density = 0.01

central_mass = 0 # Not Implemented Anymore. Keep for code compatibility.

Avg = False # Whether we average the black hole local field evaluation.
AvgDist = 1.42
CentreWeight = 1 # The relative weight of central point to the antennae.

NCV, NCW = PyULH.PInit(Avg,AvgDist,CentreWeight)  

if not Uniform:
    print('The List of Soliton Conditions is:')
    print(solitons)
    
print('The List of BH Conditions is:')  
print(particles)



# Compiling Initial Conditions

This creates a timestamped folder using the current settings, and the integrator stores all requested files in it too.

In [ ]:
run_folder = PyULH.GenerateConfig(central_mass, length, length_units, resol, duration, duration_units, step_factor, save_number, save_options, save_path, npz, npy, hdf5, s_mass_unit, s_position_unit, s_velocity_unit, solitons,start_time, m_mass_unit, m_position_unit, m_velocity_unit, particles, Uniform,Density,a, NCV,NCW)

loc = './' + save_path + '/' + run_folder

print('Compiled Config in Folder', run_folder)

Method = 1 
# 1: Real Space Linear Interpolation. 2: Fourier Sum